In [3]:
import os
import ray
if os.getcwd() != "C:/Users/Jerome/PycharmProjects/FASTgres/fastgres":
    os.chdir("C:/Users/Jerome/PycharmProjects/FASTgres/fastgres")

from workloads.workload import Workload
from query_encoding.query import Query
from query_encoding.encoded_query import EncodedQuery, EncodingInformation, Context
import baseline.utility as u
from tqdm.notebook import tqdm
import time

workload = Workload("workloads/queries/stack/", "stack")
workload.load_queries()

Loading Queries: 6191it [00:00, 48918.72it/s]


In [4]:
@ray.remote
def build_query(q, w):
    return Query(q, w)

@ray.remote
class DistEncQuery(EncodedQuery):
    def __init__(self, context: Context, query: Query, enc_inf: EncodingInformation):
        super().__init__(context, query, enc_inf)

    def encode_query(self):
        super().encode_query()
        return self.encoded_query

@ray.remote
def encode_query(context: Context, query: Query, enc_inf: EncodingInformation):
    # enc_q = EncodedQuery(context, query, enc_inf)
    dist_q = DistEncQuery.remote(context, query, enc_inf)
    enc_q = ray.get(dist_q.encode_query.remote())
    print(enc_q)
    return enc_q

In [ ]:
ray.init()

q_enc_d = dict()
q_enc_s = dict()

t0 = time.time()
for query_name in tqdm(workload.queries):
    query = build_query.remote(query_name, workload)
    q_enc_d[query_name] = query
t1 = time.time()
for query_name in tqdm(workload.queries):
    query = Query(query_name, workload)
    q_enc_s[query_name] = query
t2 = time.time()

ray.shutdown()

In [ ]:
# u.save_pickle(q_enc_d, "testing/test_dict_distributed.pkl")
# u.save_pickle(q_enc_s, "testing/test_dict_single.pkl")
print(f"Distributed / Single: {t1-t0}s / {t2-t1}s")
print(f"Speedup Factor:  {(t2-t1) / (t1-t0)}")

In [ ]:
from definitions import PG_STACK_OVERFLOW, PG_IMDB
ray.init()

query_dict = u.load_pickle("../testing/test_dict_single.pkl")
enc_inf = EncodingInformation(PG_STACK_OVERFLOW, "database_statistics/stack/", workload)
enc_inf.load_encoding_info()

# t0 = time.time()
# for query_name in tqdm(workload.queries):
#     query = query_dict[query_name]
#     context = Context()
#     context.add_context(query.context)
#     query_encoding = encode_query.remote(context, query, enc_inf)
t1 = time.time()
for query_name in tqdm(workload.queries):
    query = query_dict[query_name]
    context = Context()
    context.add_context(query.context)
    enc_q = EncodedQuery(context, query, enc_inf)
    enc_q.encode_query()
t2 = time.time()

ray.shutdown()

In [ ]:
print(f"Distributed / Single: {t1-t0}s / {t2-t1}s")
print(f"Speedup Factor:  {(t2-t1) / (t1-t0)}")

In [ ]:
ray.shutdown()

# Checking ray merging

In [6]:
query_list = list()
q_enc_s = dict()
t0 = time.time()
for query_name in tqdm(workload.queries):
    query = build_query.remote(query_name, workload)
    query_list.append(query)
vals = ray.get(query_list)
q_dict_d = dict(list(zip(workload.queries, vals)))
t1 = time.time()
for query_name in tqdm(workload.queries):
    query = Query(query_name, workload)
    q_enc_s[query_name] = query
t2 = time.time()

  0%|          | 0/6191 [00:00<?, ?it/s]

  0%|          | 0/6191 [00:00<?, ?it/s]

NameError: name 'q_enc_s' is not defined

In [ ]:
print(f"Distributed / Single: {t1-t0}s / {t2-t1}s")
print(f"Speedup Factor:  {(t2-t1) / (t1-t0)}")